In [1]:
import azureml.core
from azureml.core import Experiment, Workspace, Dataset, Datastore, ScriptRunConfig
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
import os
import shutil
import glob
import math
from os.path import join
import tensorflow as tf
from sys import path
import numpy as np

# check core SDK version number

print("Azure ML SDK Version: ", azureml.core.VERSION)

Azure ML SDK Version:  1.18.0


In [2]:
# load workspace configuration from the config.json file in the current folder.
ws = Workspace.from_config()
# get metadata about the workspace
print(ws.name, ws.location, ws.resource_group, sep='\t')
# list the registered datastores
ws.datastores

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code EB26D5LD7 to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.
solar-compviz	eastus2	PLACE


WARNING - Failed to authenticate to tenant '72f988bf-86f1-41af-91ab-2d7cd011db47' due to error 'Get Token request returned http error: 400 and server response: {"error":"interaction_required","error_description":"AADSTS50076: Due to a configuration change made by your administrator, or because you moved to a new location, you must use multi-factor authentication to access '797f4846-ba00-4fd7-ba43-dac1f8f63013'.\r\nTrace ID: f4818396-eabb-4acd-8bf4-96757f52a200\r\nCorrelation ID: 0295cc35-2426-434b-9173-da8f469812bd\r\nTimestamp: 2021-11-24 02:01:41Z","error_codes":[50076],"timestamp":"2021-11-24 02:01:41Z","trace_id":"f4818396-eabb-4acd-8bf4-96757f52a200","correlation_id":"0295cc35-2426-434b-9173-da8f469812bd","error_uri":"https://login.microsoftonline.com/error?code=50076","suberror":"basic_action"}'.Will continue to look for other tenants to find subscriptions to which you have access


{'workspaceartifactstore': {
   "name": "workspaceartifactstore",
   "container_name": "azureml",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'solardatafilestore': {
   "name": "solardatafilestore",
   "container_name": "gee-data",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspaceblobstore': {
   "name": "workspaceblobstore",
   "container_name": "azureml-blobstore-20200d5f-4836-4a79-bf1c-7bd46e0d0c35",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 },
 'workspacefilestore': {
   "name": "workspacefilestore",
   "container_name": "azureml-filestore-20200d5f-4836-4a79-bf1c-7bd46e0d0c35",
   "account_name": "solarcompviz6161005396",
   "protocol": "https",
   "endpoint": "core.windows.net"
 }}

In [3]:
# access our registered data share containing image data in this workspace
datastore = Datastore.get(workspace = ws, datastore_name = 'solardatafilestore')

cpk_train_path = (datastore, 'CPK_solar/training/')
cpk_eval_path = (datastore, 'CPK_solar/eval/')

nc_train_path = (datastore, 'NC_solar/training/')
nc_eval_path = (datastore, 'NC_solar/eval/')

test_path = (datastore, 'CPK_solar/predict/testpred6')

# cpk_train_dataset = Dataset.File.from_files(path = [cpk_train_path])
# cpk_eval_dataset = Dataset.File.from_files(path = [cpk_eval_path])

# nc_train_dataset = Dataset.File.from_files(path = [nc_train_path])
# nc_eval_dataset = Dataset.File.from_files(path = [nc_eval_path])

# when we combine datasets the selected directories and relative paths to the datastore are brought in
# mount folder
# |-cddatafilestore
# | |-GEE
# | | |-training
# | | |-eval
# | |-Onera
# | | |-training
# | | |-eval
train_dataset = Dataset.File.from_files(path = [cpk_train_path, nc_train_path])
eval_dataset = Dataset.File.from_files(path = [cpk_eval_path, nc_eval_path])
test_dataset = Dataset.File.from_files(path = [test_path])

In [14]:
experiment_name = 'solar-nc-cpk'
exp = Experiment(workspace = ws, name = experiment_name)

In [10]:
# define the compute target
ws.compute_targets
compute_target = ws.compute_targets['mevans1']

In [11]:
envs = Environment.list(workspace = ws)
env = envs.get('solar-training')

In [7]:
%cd Satellite_ComputerVision
!git pull
%cd ..

/mnt/batch/tasks/shared/LS_root/mounts/clusters/mevans1/code/Users/mevans/Satellite_ComputerVision
remote: Enumerating objects: 27, done.
remote: Counting objects: 100% (27/27), done.
remote: Compressing objects: 100% (10/10), done.
remote: Total 20 (delta 16), reused 14 (delta 10), pack-reused 0
Unpacking objects: 100% (20/20), 3.06 KiB | 0 bytes/s, done.
From https://github.com/mjevans26/Satellite_ComputerVision
   28f7598..96f77f7  master     -> origin/master
Updating 28f7598..96f77f7
Fast-forward
 .spyproject/workspace.ini |   2 +-
 azure/train_acd.py        |   9 ++--
 utils/prediction_tools.py | 123 +++++++++++++++++++++++++++-------------------
 3 files changed, 80 insertions(+), 54 deletions(-)
/mnt/batch/tasks/shared/LS_root/mounts/clusters/mevans1/code/Users/mevans


In [8]:
# attach our utilities folder to the path to import modules
path.append('/home/azureuser/cloudfiles/code/Users/mevans/Satellite_ComputerVision/utils')

In [9]:
from model_tools import get_model, make_confusion_matrix
from processing import get_training_dataset

In [8]:
# Define some global variabes

BANDS = ['B2', 'B3', 'B4', 'B8', 'B11', 'B12']
RESPONSE = 'landcover'
FEATURES = BANDS + [RESPONSE]

# Specify the size and shape of patches expected by the model.
KERNEL_SIZE = 256
KERNEL_SHAPE = [KERNEL_SIZE, KERNEL_SIZE]
COLUMNS = [
  tf.io.FixedLenFeature(shape=KERNEL_SHAPE, dtype=tf.float32) for k in FEATURES
]
FEATURES_DICT = dict(zip(FEATURES, COLUMNS))

In [10]:
# Create mountcontext and mount the dataset
test_mount = train_dataset.mount()

# open access to the mount point
test_mount.start() 

# Get the mount point
dataset_mount_folder = test_mount.mount_point
print(dataset_mount_folder)

test_files = []
for root, dirs, files in os.walk(dataset_mount_folder):
    for f in files:
        test_files.append(join(root, f))

# test_files = glob.glob(join(dataset_mount_folder, '*.'))

# read in the training data so we can calculate size

training = get_training_dataset(
    files = test_files,
    ftDict = FEATURES_DICT,
    features = BANDS,
    response = RESPONSE,
    buff = 1,
    batch = 1,
    repeat = False)

/tmp/tmptskm6c4k
CRITICAL - MSI: Failed to retrieve a token from 'http://127.0.1.1:46808/MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01' with an error of 'HTTPConnectionPool(host='127.0.1.1', port=46808): Max retries exceeded with url: /MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa6d4744c50>: Failed to establish a new connection: [Errno 111] Connection refused',))'.


In [ ]:
iterator = iter(training)
print(iterator.next())

In [12]:
# # calculate some summary statistics used in model training
m = get_model(depth = len(BANDS), optim = tf.keras.optimizers.Adam(learning_rate = 0.001, beta_1=0.9, beta_2=0.999), loss = 'mse', mets = [tf.keras.metrics.categorical_accuracy], bias = None)
train_con_mat = make_confusion_matrix(training, m)
classums = train_con_mat.sum(axis = 1)
BIAS = np.log(classums[1]/classums[0])
WEIGHT = classums[0]/classums[1]
TRAIN_SIZE = train_con_mat.sum()//(256*256)

test_mount.stop()

CRITICAL - MSI: Failed to retrieve a token from 'http://127.0.1.1:46808/MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01' with an error of 'HTTPConnectionPool(host='127.0.1.1', port=46808): Max retries exceeded with url: /MSI/auth/?resource=https://management.core.windows.net/&api-version=2017-09-01 (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fa6d475bda0>: Failed to establish a new connection: [Errno 111] Connection refused',))'.
1/1 [==============================] - 0s 384ms/sample
7755


In [13]:
print('size', TRAIN_SIZE)
print('weight', WEIGHT)
print('bias', BIAS)

size 7755
weight 78.48739612112549
bias -4.362938052930838


In [6]:
# use the azure folder as our script folder
source = 'Satellite_ComputerVision'
util_folder = 'utils'
script_folder = f'{source}/azure'
script_file = 'train_solar.py'

In [ ]:
# copy the entire 'utils' folder from github repo to local
# shutil.copytree(src = f'/home/azureuser/cloudfiles/code/Users/mevans/Satellite_ComputerVision/{script_folder}',
# dst = f'/home/azureuser/cloudfiles/code/Users/mevans/{script_folder}')

# shutil.copytree(src = f'/home/azureuser/cloudfiles/code/Users/mevans/Satellite_ComputerVision/{util_folder}',
# dst = f'/home/azureuser/cloudfiles/code/Users/mevans/{script_folder}/{util_folder}')

In [12]:
# create script run config
# WEIGHT = int(math.ceil(WEIGHT/5)*5)
# define the command line arguments to our training sript
args = [
    '--train_data', train_dataset.as_mount(),
    '--eval_data', eval_dataset.as_mount(),
    '--test_data', test_dataset.as_mount(),
    '--weight', 40,
    '--bias', -4,#BIAS,
    '-lr', 0.001,
    '--epochs', 100,
    '--batch', 16,
    '--size', 7755,
    '--kernel_size', 256,
    '--response', f'{RESPONSE}']
    # '--bands', 'B2' 'B3' 'B4' 'B8' 'B2_1' 'B3_1' 'B4_1' 'B8_1',
    # '--splits', None]

# TODO: figure out how to pass lists as command line arguments

src = ScriptRunConfig(source_directory=script_folder,
                      script=script_file, 
                      arguments=args,
                      compute_target=compute_target,
                      environment=env)

In [15]:
#  run the training job
run = exp.submit(config=src, tags = dict({'splits':'None', 'model':'Unet', 'dataset':'CPK+NC', 'normalization':'pixel', 'epochs':'0-100'}))
run

In [14]:
from azureml.widgets import RunDetails
RunDetails(run).show()

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…